In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library
import wikipedia as wp
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Libraries imported.


In [2]:
df=pd.read_csv("./tor_geoinfo.csv")
df.drop(['Unnamed: 0', 'Postcode'], axis=1, inplace=True)
print(df.shape)
df.head()

(210, 4)


,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,North York,Lawrence Heights,43.718518,-79.464763
4,North York,Lawrence Manor,43.718518,-79.464763


In [3]:
html2 = wp.page("Demographics of Toronto neighbourhoods").html().encode("UTF-8")
dft = pd.read_html(html2)[1]
dft = pd.DataFrame(dft)

html1 = wp.page("Boroughs of New York City").html().encode("UTF-8")
dfny = pd.read_html(html1)[0]
dfn = pd.DataFrame(dfny)
dfn.drop(dfn.index[5:], inplace=True)

In [4]:
dft.head()

,Name,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map
0,Toronto CMA Average,NaN,All,5113149,5903.63,866,9.0,40704,10.6,11.4,NaN,NaN,NaN
1,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...",44577,12.45,3580,4.6,25750,11.1,5.9,Cantonese (19.3%),19.3% Cantonese,NaN
2,Alderwood,E,"0211.00, 0212.00",11656,4.94,2360,-4.0,35239,8.8,8.5,Polish (6.2%),06.2% Polish,NaN
3,Alexandra Park,OCoT,0039.00,4355,0.32,13609,0.0,19687,13.8,28.0,Cantonese (17.9%),17.9% Cantonese,NaN
4,Allenby,OCoT,0140.00,2513,0.58,4333,-1.0,245592,5.2,3.4,Russian (1.4%),01.4% Russian,NaN


In [5]:
print(dfn.shape)
dfn.head()

(5, 9)


New York City's five boroughsvte                               \
                      Jurisdiction                   Population   
                           Borough    County Estimate (2017)[3]   
0                        The Bronx     Bronx            1471160   
1                         Brooklyn     Kings            2648771   
2                        Manhattan  New York            1664727   
3                           Queens    Queens            2358582   
4                    Staten Island  Richmond             479458   

                                                                \
  Gross Domestic Product                    Land area            
        billions(US$)[4] per capita(US$) square miles squarekm   
0                 28.787           19570        42.10   109.04   
1                 63.303           23900        70.82   183.42   
2                629.682          378250        22.83    59.13   
3                 73.842           31310       108.53   281.09   
4                 11.249           23460        58.37   151.18   

                                 
           Density               
  persons / sq. mi persons /km2  
0            34653        13231  
1            37137        14649  
2            72033        27826  
3            21460         8354  
4             8112         3132

#### NYC datasets

In [6]:
df_jurisdiction=dfn['New York City\'s five boroughsvte']['Jurisdiction']
df_population = dfn['New York City\'s five boroughsvte']['Population']
df_gdp = dfn['New York City\'s five boroughsvte']['Gross Domestic Product']
df_land = dfn['New York City\'s five boroughsvte']['Land area']['squarekm']
df_density = dfn['New York City\'s five boroughsvte']['Density']['persons /km2']

In [7]:
df_nyc = pd.DataFrame(columns=['borough', 'population', 'income_per_capita', 'land_area(km2)', 'density(person/km2)'])
df_nyc['borough']=df_jurisdiction['Borough']
df_nyc['population'] = df_population['Estimate (2017)[3]']
df_nyc['income_per_capita'] = df_gdp['per capita(US$)']
df_nyc['land_area(km2)'] = df_land
df_nyc['density(person/km2)']=df_density
df_nyc['location']=['nyc']*(df_nyc.shape[0])

#### Toronto datasets

In [8]:
df_toronto = pd.DataFrame(columns=['neighborhoods', 'population', 'income_per_capita', 'land_area(km2)', 'density(person/km2)'])
df_toronto['neighborhoods'] = dft['Name']
df_toronto['population'] = dft['Population']
df_toronto['income_per_capita'] = dft['Average Income']
df_toronto['land_area(km2)'] = dft['Land area (km2)']
df_toronto['density(person/km2)'] = dft['Density (people/km2)']
df_toronto['location']=['toronto']*(df_toronto.shape[0])
df_toronto.drop([0], axis=0, inplace=True)

In [9]:
df_toronto.head(20)

,neighborhoods,population,income_per_capita,land_area(km2),density(person/km2),location
1,Agincourt,44577,25750,12.45,3580,toronto
2,Alderwood,11656,35239,4.94,2360,toronto
3,Alexandra Park,4355,19687,0.32,13609,toronto
4,Allenby,2513,245592,0.58,4333,toronto
5,Amesbury,17318,27546,3.51,4934,toronto
6,Armour Heights,4384,116651,2.29,1914,toronto
7,Banbury,6641,92319,2.72,2442,toronto
8,Bathurst Manor,14945,34169,4.69,3187,toronto
9,Bay Street Corridor,4787,40598,0.11,43518,toronto
10,Bayview Village,12280,46752,4.14,2966,toronto


In [10]:
df_nyc

,borough,population,income_per_capita,land_area(km2),density(person/km2),location
0,The Bronx,1471160,19570,109.04,13231,nyc
1,Brooklyn,2648771,23900,183.42,14649,nyc
2,Manhattan,1664727,378250,59.13,27826,nyc
3,Queens,2358582,31310,281.09,8354,nyc
4,Staten Island,479458,23460,151.18,3132,nyc
